In [1]:
import dotenv

dotenv.load_dotenv(dotenv.find_dotenv())

from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import AzureChatOpenAI
from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from pathlib import Path
from tqdm import tqdm
from typing import Dict, Optional, List
import random

In [2]:
llm = AzureChatOpenAI(
    temperature=0.0,
    azure_deployment="gpt4o",
    openai_api_version="2023-07-01-preview",
)

In [3]:
class TrashItem(BaseModel):
    """
    A trash item that needs to be sorted into a category.
    """

    item_id: Optional[str] = Field(description="The unique identifier of the item.")
    direction: Optional[str] = Field(
        description="How to dispose of the item according to the rules."
    )


class Quirk(BaseModel):
    """
    A local rule or quirk relevant to the person trying to sort their garbage.
    Should include a concrete fine, specific sorting rule, collection day of the week, etc.
    Should not include general information like "fines may apply" or "state plans to enforce this law by 2025".
    Should not include rules that apply to facilities or businesses, only to individuals.
    """

    description: Optional[str] = Field(
        description="A description of the rule or quirk."
    )


class SortingInfo(BaseModel):
    """
    Information about how to sort a list of trash items.
    """

    sorted_items: Optional[List[TrashItem]] = Field(
        description="A list of trash items and their corresponding categories."
    )
    local_quirks: Optional[List[Quirk]] = Field(
        description="Local quirks that need to be taken into account when sorting the items."
    )


In [4]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert extraction algorithm designed to help with sorting garbage items based on local waste disposal rules. "
            "You will receive a list of garbage item names and need to assign each one to its corresponding category according to the provided disposal rules. "
            "Only extract relevant information from the text. "
            "If you do not know the value of an attribute asked to extract, return null for the attribute's value.",
        ),
        # ↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓
        MessagesPlaceholder("examples"),  # <-- EXAMPLES!
        # ↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑
        ("human", "{rules}\n===\n{items}"),
    ]
)

prompt.invoke(
    {
        "rules": "some rules",
        "items": "this is items",
        "examples": [HumanMessage(content="testing 1 2 3")],
    }
)

ChatPromptValue(messages=[SystemMessage(content="You are an expert extraction algorithm designed to help with sorting garbage items based on local waste disposal rules. You will receive a list of garbage item names and need to assign each one to its corresponding category according to the provided disposal rules. Only extract relevant information from the text. If you do not know the value of an attribute asked to extract, return null for the attribute's value."), HumanMessage(content='testing 1 2 3'), HumanMessage(content='some rules\n===\nthis is items')])

In [5]:
import uuid
from typing import Dict, List, TypedDict

from langchain_core.messages import (
    AIMessage,
    BaseMessage,
    HumanMessage,
    SystemMessage,
    ToolMessage,
)
from langchain_core.pydantic_v1 import BaseModel, Field


class Example(TypedDict):
    """A representation of an example consisting of text input and expected tool calls.

    For extraction, the tool calls are represented as instances of pydantic model.
    """

    input: str  # This is the example text
    tool_calls: List[BaseModel]  # Instances of pydantic model that should be extracted


def tool_example_to_messages(example: Example) -> List[BaseMessage]:
    """Convert an example into a list of messages that can be fed into an LLM.

    This code is an adapter that converts our example to a list of messages
    that can be fed into a chat model.

    The list of messages per example corresponds to:

    1) HumanMessage: contains the content from which content should be extracted.
    2) AIMessage: contains the extracted information from the model
    3) ToolMessage: contains confirmation to the model that the model requested a tool correctly.

    The ToolMessage is required because some of the chat models are hyper-optimized for agents
    rather than for an extraction use case.
    """
    messages: List[BaseMessage] = [HumanMessage(content=example["input"])]
    tool_calls = []
    for tool_call in example["tool_calls"]:
        tool_calls.append(
            {
                "id": str(uuid.uuid4()),
                "args": tool_call.dict(),
                # The name of the function right now corresponds
                # to the name of the pydantic model
                # This is implicit in the API right now,
                # and will be improved over time.
                "name": tool_call.__class__.__name__,
            },
        )
    messages.append(AIMessage(content="", tool_calls=tool_calls))
    tool_outputs = example.get("tool_outputs") or [
        "You have correctly called this tool."
    ] * len(tool_calls)
    for output, tool_call in zip(tool_outputs, tool_calls):
        messages.append(ToolMessage(content=output, tool_call_id=tool_call["id"]))
    return messages

In [6]:
with open("korea_summary.txt", "r") as f:
    example_rules = f.read()

example_items = [
    {"item_id": 0, "name": "plastic"},
    {"item_id": 1, "name": "paper"},
    {"item_id": 2, "name": "foam"},
    {"item_id": 3, "name": "food"},
]
example_input = f"{example_rules}\n===\n{example_items}"


example_output = SortingInfo(
    sorted_items=[
        TrashItem(item_id="0", direction="Recyclable Waste (재활용 쓰레기) (No special bags required)"),
        TrashItem(item_id="1", direction="Recyclable Waste (재활용 쓰레기) (No special bags required)"),
        TrashItem(item_id="2", direction="General Waste Bag (일반 쓰레기 봉투)"),
        TrashItem(item_id="3", direction="Food Waste Bag (음식물 쓰레기 봉투)"),
    ],
    local_quirks=[
        Quirk(
            description="Make sure to put items in appropriate garbage bags. Garbage bags are color-coded and district-specific. You can get them at local convenience stores."
        ),
        Quirk(description="Penalty for non-compliance is a fine of up to 300,000 KRW."),
        Quirk(description="Do not put egg shells, seafood shells, and animal bones into the Food Waste Bag."),
    ],
)

examples = [
    (
        example_input,
        example_output,
    )
]

messages = []

for text, tool_call in examples:
    messages.extend(
        tool_example_to_messages({"input": text, "tool_calls": [tool_call]})
    )

example_prompt = prompt.invoke(
    {"rules": "some rules", "items": "this is items", "examples": messages}
)

for message in example_prompt.messages:
    print(f"{message.type}: {message}")

system: content="You are an expert extraction algorithm designed to help with sorting garbage items based on local waste disposal rules. You will receive a list of garbage item names and need to assign each one to its corresponding category according to the provided disposal rules. Only extract relevant information from the text. If you do not know the value of an attribute asked to extract, return null for the attribute's value."
human: content='In this community, garbage sorting rules are as follows:\n\n### General Waste Bag (일반 쓰레기 봉투)\n- **Color:** Typically white or green (varies by district).\n- **Contents:** Everything not classified as recyclable or food waste. Examples include used tissues, used toilet paper (when not flushed), sanitary pads, old shoes, and clothes.\n\n### Food Waste Bag (음식물 쓰레기 봉투)\n- **Contents:** Edible waste such as fruit peels, vegetable peels, uneaten meat, and raw eggs (without the shell).\n- **Exceptions:** Egg shells, crustacean shells, clam shells, 

In [7]:
runnable = prompt | llm.with_structured_output(schema=SortingInfo)

In [8]:
with open("california_summary.txt") as f:
    rules = f.read()

item_names = [
    "plastic",
    "glass",
    "paper",
    "carton",
    "metal",
    "foam",
    "special",
    "food",
    "general",
]


def get_random_items():
    random_count = random.randint(1, 7)
    random_items = [
        {"item_id": i, "name": item_names[random.randint(0, len(item_names) - 1)]}
        for i in range(random_count)
    ]
    return random_items

In [9]:
items = get_random_items()
output = runnable.invoke({"rules": rules, "items": items, "examples": messages})

In [10]:
items

[{'item_id': 0, 'name': 'general'},
 {'item_id': 1, 'name': 'glass'},
 {'item_id': 2, 'name': 'food'},
 {'item_id': 3, 'name': 'metal'},
 {'item_id': 4, 'name': 'glass'},
 {'item_id': 5, 'name': 'paper'},
 {'item_id': 6, 'name': 'paper'}]

In [11]:
output.dict()

{'sorted_items': [{'item_id': '0',
   'direction': 'Gray Container (Non-organic and non-recyclable waste)'},
  {'item_id': '1', 'direction': 'Blue Container (Traditional recyclables)'},
  {'item_id': '2', 'direction': 'Green Container (Food waste)'},
  {'item_id': '3', 'direction': 'Blue Container (Traditional recyclables)'},
  {'item_id': '4', 'direction': 'Blue Container (Traditional recyclables)'},
  {'item_id': '5', 'direction': 'Blue Container (Traditional recyclables)'},
  {'item_id': '6', 'direction': 'Blue Container (Traditional recyclables)'}],
 'local_quirks': [{'description': 'Non-compliance with sorting and disposal regulations may result in fines.'},
  {'description': 'Gray container waste must be transported to a high diversion organic waste processing facility.'},
  {'description': 'Uncontainerized green waste and yard waste can be collected in the streets if requirements are met.'}]}